In [104]:
import matplotlib.pyplot as plt
from graphics import *
from operator import *
from random import *
#from visual import *
import numpy as np
import timeit
from math import *

In [105]:
def initialize(speed = 1, N = 100, width = 1000, height = 1000):
    """
    Initializes our agent set with randomly directed speeds, draws the window and the agents
    """
    agents = [Point(width*random(), height*random()) for i in range(N)]
    speeds = [np.array([0.0, 0.0]) for i in range(N)]
    for i in range(N):
        theta = 2 * np.pi * random()
        speeds[i][0] = speed * np.cos(theta)
        speeds[i][1] = speed * np.sin(theta)

    return agents, speeds, draw_agents(agents, width, height)


def draw_agents(agents, width, height):
    win = GraphWin("Swarm", width, height) # size of box
    for agent in agents:
        agent.draw(win)
    win.getMouse()
    return win


def couple_speeds(agents, speeds, a, s, N):
    """
    Simplest model: for each agent, it will give its nearest neighbour a fraction of its speed and re-normalize it
    
    """
    nearest_neighbours = [nearest_neighbour(agent, agents, N) for agent in agents]
    for i in range(N):
            weightedSpeed = a * speeds[nearest_neighbours[i]]
            speeds[i] = speeds[i] + weightedSpeed
            speeds[i] = s * normalized(speeds[i]) ### PROBLEMA?-------------------------------------------------


def get_distances(agent, agents):
    """
    Given one angent and the set of all agents, 
    computes the distances from the first to all of the others
    """
    
    dists = [(a.getX() - agent.getX())**2 + (a.getY() - agent.getY())**2 for a in agents]
    for i in range(len(dists)):
        if dists[i] == 0:
            dists[i] = 0.1
    return dists


def nearest_neighbour(agent, agents, N):
    """
    Returns the index for the agent with smallest Eucledian distance to agent in question
    """
    distances = get_distances(agent, agents)
    j = next(i for i in range(N) if agents[i] is agent)
    distances[j] = distances[j-1] + 1

    return distances.index(min(distances))


def treat_boundary(x_bound, y_bound, agents, speeds, N):
    for i in range(N):
        [dx, dy] = [0, 0]
        [x, y] = [agents[i].getX(), agents[i].getY()]
        if x > x_bound:
            speeds[i][0] = -speeds[i][0]
            dx = x_bound - x         
        
        elif x < 0:
            speeds[i][0] = -speeds[i][0]
            dx = -x
        
        if y > y_bound:
            speeds[i][1] = -speeds[i][1]
            dy = y_bound - y
        
        elif y < 0:
            speeds[i][1] = -speeds[i][1]
            dy = -y

        agents[i].move(dx, dy)
        
        
def periodic_boundary(x_bound, y_bound, agents, speeds, N):  #Changed from rigid boundaries do periodic boundary condition
    [dx, dy] = [0, 0]
    for i in range(N):
        [x, y] = [agents[i].getX(), agents[i].getY()]
        if x > x_bound:
            dx = -x_bound         
        
        elif x < 0:
            dx = x_bound
        
        if y > y_bound:
            dy = -y_bound
        
        elif y < 0:
            dy = y_bound

        agents[i].move(dx, dy)

def next_step(agents, speeds, dt, N):
    dxvec = [dt * speeds[i][0] for i in range(N)]
    dyvec = [dt * speeds[i][1] for i in range(N)]
    for i in range(N):
        agents[i].move(dxvec[i], dyvec[i])
        

def normalized(vector):
    if vector[0] == vector[1] == 0:
        return vector
    return vector / np.linalg.norm(vector)


#def get_v_avg(speeds, N):
#    return
#   
#def get_potencials(distances, N):
#    potentials = np.zeros(N, N)
#    for i in range(N):
#        for j in range(i+1,N):
#            potentials[i][j] = potentials[j][i] = morse_pot(distances[i][j])
#    return potentials  
#    
#def morse_pot(r, ca = 0.5, cr = 1, la = 3, lr = 0.5):
#    return cr*np.exp(-r/lr) - ca * np.exp(-r)
#
#def weird_paper(agents, N, dt):
#    distances = [get_distances(agent, agents) for agent in agents]
#    potencials = get_potencials(distances, N)
#    for i in range(N):
#            newSpeed = speed[i] + dt * potentials
#            speeds[i] = speeds[i] + weightedSpeed
#            speeds[i] = s * normalized(speeds[i]) ### PROBLEMA?-------------------------------------------------
#
#    return

In [154]:
def couzin(agents, speeds, N, s, noise, rr, ro, ra):
    # watch only particles inrepuls
    for i in range(N): ### FIX - Eliminate, in some way the i-i interaction
        curr_agent = agents[i]
        distances = get_distances(curr_agent, agents)
        r_dir = np.array([0.0, 0.0])
        o_dir = np.array([0.0, 0.0])
        a_dir = np.array([0.0, 0.0])
        repulsion_flag = False    
        
        for j in range(N):
            if i == j:
                continue

            if distances[j] < rr:
                temp_vec = np.array([agents[j].getX() - curr_agent.getX(), agents[j].getY() - curr_agent.getY()])
                temp_vec = normalized(temp_vec)
                r_dir = r_dir + temp_vec
                repulsion_flag = True
            
            elif not repulsion_flag:

                if distances[j] < ro:
                    o_dir = o_dir + speeds[j]
                
                elif distances[j] < ra:
                    temp_vec = np.array([agents[j].getX() - curr_agent.getX(), agents[j].getY() - curr_agent.getY()])
                    temp_vec = normalized(temp_vec)
                    a_dir = a_dir + temp_vec
            
        #Out of for (j), we treat now the resulting direction vector
        if repulsion_flag:
            tot_dir =  normalized(-r_dir)
        else:
            o_dir = normalized(o_dir)
            a_dir = normalized(a_dir)
            tot_dir = o_dir + a_dir
            tot_dir = normalized(tot_dir)
            
        tot_dir = normalized(normalized(tot_dir) + noise * np.array([(-1)+2*random(),(-1)+2*random()]))
        
        #avoid pts stoping when not interacting
        if np.linalg.norm(tot_dir) != 0:
            speeds[i] = s*tot_dir 

def biaser(speeds, N, s, i, prop, bias, dev_bias, rot_bias, weight):
    #bias = np.array([0.0,1.0])
    #Ns has to be integer 
    Ns = int(N * prop)
    gbias = np.random.normal(bias, dev_bias, )
    for i in range(Ns):
        tot_dir = normalized(normalized(speeds[i]) + weight * gbias)
        
        if np.linalg.norm(tot_dir) != 0:
            speeds[i] = s*tot_dir
    #bias = np.dot(tot_dir,np.array([[np.cos(rot_bias*i), 0],[0, np.sin(rot_bias*i)]]))
    return bias

def quality (agents, speeds, N, bias, window, old_cm):
#ACCURACY (DIRECTIONS DISTRIBUTION)
    dev = 0.0
    for i in range(N):
        dev += np.arccos(np.dot(bias,normalized(speeds[i])))
    dev_avg = dev/(N*2*(np.pi))
    #print (dev_avg)
    
#ELONGATION: SHAPE OF SWARM
    #can be smarter if we make agents become poses before, more globally in  the code
    poses = [np.array([a.getX(), a.getY()]) for a in agents]
    #Center of Mass
    cm = np.mean(poses,axis=0)
    #Standard Deviation
    std = np.std(poses,axis=0)
    #print (cm, std)

    #Elongation >1 means 
    elong = std[1] / std[0]
    #Drawing it
    #old_cm = Point(cm[0],cm[1])
    #old_cm.draw(window)
    #old_cm.setFill("red")

#GROUP DIRECTION
    #Vector
    group_dir = np.array([cm[0] - old_cm[0], cm[1] - old_cm[1]])                
    print (group_dir)
    # Norm
    group_dir = np.linalg.norm(group_dir)
    print (group_dir)
    return cm
    
def vicsek(agents, speeds, N, s, noise, r): # s=speed, noise= letter csi temperature factor, r=radius of interaction
    # consider only particles within 'r' from pt_i, align pt_i with v_avg
    for i in range(N):
        distances = get_distances(agents[i], agents)
        tot_dir = np.array([0.0, 0.0])
        
        for j in range(N):
            if distances[j] < r:
                tot_dir = tot_dir + speeds[j]

        tot_dir = s * normalized(normalized(tot_dir) + noise * np.array([(-1)+2*random(),(-1)+2*random()]))
        if np.linalg.norm(tot_dir) != 0:
            speeds[i] = tot_dir

def gueron():
    return

In [157]:
def simulate(N_steps, a, dt, N, width, height, s, noise, prop, weight, bias, dev_bias, rot_bias):
    """
    Simulates motion of swarm. Recieves following parameters:

    N_steps  - number of steps to perform
    a -  coupling between neighbouring points
    dt - time step to be used 
    N - number of points to be used
    width & heigth of window
    s - module of speed throughout agents

    """
    #Watch out!! format of old_cm was giving problems
    #old_cm=Point(0,0)
    #old_cm.setFill("red")
    old_cm=np.array([0.0,0.0])
    
    
    agents, speeds, window = initialize(s, N, width, height)
    for i in range(N_steps):
        next_step(agents, speeds, dt, N)
        import timeit

        start = timeit.default_timer()
        
        ## MODEL CHOSEN
        #couple_speeds(agents, speeds, a, s, N)
        #vicsek(agents, speeds, N, s, 0.1, r = 500)
        couzin(agents, speeds, N, s, noise,200,2500,3000)
        #EXTRA ON COUZIN MODEL
        bias = biaser(speeds, N, s, i, prop, bias, dev_bias, rot_bias, weight)
        #INFORMATION TRANSFER: SHAPE & DIRECTION & ALIGNMENT QUALITY 
        #point_cm.undraw()
        old_cm = quality(agents, speeds, N, bias, window, old_cm)
        #print (old_cm)
        
        ## BOUNDARY CONDITIONS
        #treat_boundary(width, height, agents, speeds, N)
        periodic_boundary(width, height, agents, speeds, N)

        #time.sleep(0.02)
        stop = timeit.default_timer()
        #print stop - start
    window.close()
    

#simulate(N_steps, a, dt, N, width, height, s, noise, prop, weight, bias, dev_bias, rot_bias)
simulate(3000, 0.1, 2, 50, 500, 500, 2, 0.01, 0.4,0.5, np.array([0.0,-1.0]), 0.1, 1)

[ 237.6960397  237.2967082]
335.870711751
[ -7.85459695e-02  -1.09977063e+02]
109.977091276
[ 379.94660384   -9.52747252]
380.066039663
[-370.19742115 -110.76096557]
386.41185556
[   0.26954547  219.52148268]
219.521648164
[-149.44670571   -0.50711576]
149.447566105
[ 140.69935389 -340.80846583]
368.709531432
[ 150.70003975  329.43195414]
362.264978139
[-139.43220323   -0.65939175]
139.433762397
[ 0.16316038 -0.9659802 ]
0.979662727133
[-0.03829756 -1.12354409]
1.12419661549
[ 0.25512574 -1.20764016]
1.23429489977
[ 0.34395793 -1.37153974]
1.41401142534
[   0.59612644  358.7192844 ]
358.719779727
[   0.90104438 -290.77235518]
290.773751258
[   1.34262865  278.96645788]
278.969688812
[   0.82712434 -310.68767253]
310.688773532
[   1.11076519  299.1395709 ]
299.141633145
[   0.77888913 -290.72975858]
290.730801935
[   1.10570936 -200.94721691]
200.950258962
[   1.19807121  209.25897303]
209.262402663
[   1.14503801  369.11927917]
369.121055165
[   0.78462566 -381.26656335]
381.267370712


[-0.3886657 -0.7231674]
0.820994588092
[-0.81232021 -0.58827493]
1.00296137323
[  -0.59448414  379.38293125]
379.383397025
[  -0.68242102 -371.8224049 ]
371.823031138
[-0.36532726 -1.4406333 ]
1.48623293116
[-0.52279584 -1.09009307]
1.20897410434
[-1.28537189 -0.83462311]
1.53257190023
[-0.93139295 -1.09149043]
1.43486730596
[  -1.21424426  239.27716963]
239.280250529
[  -0.51338378 -230.50338438]
230.503956089
[-1.02126458  0.00826512]
1.02129802163
[-2.02111461 -0.79943139]
2.17347528539
[-1.37354779 -1.36874708]
1.93909826806
[-0.94366785 -0.93181339]
1.32619199343
[-0.98828249 -1.3656368 ]
1.68572422496
[-0.44544595 -1.53727911]
1.60051528041
[ -0.34956029  18.87723201]
18.8804682348
[  0.48779608 -41.20229734]
41.2051847629
[ 149.77943054  239.08911849]
282.130261388
[-140.08271735 -231.90906594]
270.933539019
[ 0.03404328 -1.47633728]
1.47672973571
[ 0.22431052 -1.12819486]
1.15027773052
[ 0.55817562 -0.7544537 ]
0.938488365468
[ 0.53325986 -0.98492512]
1.1200194538
[ -3.55199453

[  -0.66603366  108.28752827]
108.289576505
[  -0.80547308 -101.96042396]
101.963605477
[  -0.18943098  178.52388678]
178.523987287
[  4.49811773e-02  -1.91178357e+02]
191.17836234
[ -1.66804684e-02  -4.12963848e+01]
41.296388137
[ -0.36369209 -31.03793221]
31.0400629501
[  -0.67173172  218.72455324]
218.725584721
[  -0.40928245 -170.55730873]
170.557799799
[ -0.33908323 -50.216735  ]
50.2178798019
[  -0.56092261  219.55736508]
219.558081596
[ -1.55552011e-01  -2.80834216e+02]
280.834259489
[  4.53230368e-02   2.98207147e+02]
298.207150188
[  -0.35988556 -232.16100558]
232.161284518
[  -0.50968784  157.76040824]
157.761231579
[  -0.23790402 -101.8662341 ]
101.866511907
[  -0.32912501  108.36032212]
108.360821951
[ -1.29281114e-01  -1.81954791e+02]
181.954836443
[  0.15512231  67.86184685]
67.8620241473
[  1.04880638e-01   1.07611674e+02]
107.611725408
[-0.0854166  -1.31949083]
1.32225263986
[ -0.03357184  28.48333379]
28.4833535703
[ -0.5407037  -41.18754936]
41.1910983435
[ 239.699986

[ 0.4642801   0.65597908]
0.803656990715
[ 0.54802689 -0.77732703]
0.951089256888
[ 0.25131271 -0.87044981]
0.906002734101
[ 0.44948354 -1.55580104]
1.61942963419
[ 0.71617277 -0.7406615 ]
1.03028292294
[ 0.16875719 -0.9543343 ]
0.969140315352
[ 0.40382343 -0.90140519]
0.987727023012
[-0.02858917 -0.58052079]
0.581224338993
[-0.48864592 -1.20492569]
1.30023872994
[-0.90146251 -1.7063961 ]
1.92987624301
[-0.07842955 -1.39545002]
1.39765230515
[-0.38966744 -0.85972204]
0.943908206168
[-0.0523804   0.24026212]
0.245905655102
[ 0.35080105 -0.82577106]
0.897195198407
[ 0.31311168 -0.74534474]
0.808441529957
[ 0.70473042 -1.07352606]
1.28417412225
[ 1.37495712 -0.93346007]
1.66188290174
[ 1.29573854 -0.38232408]
1.35096634544
[ 1.18527088 -0.31314496]
1.22593915925
[ 0.8252093  -0.58523715]
1.01166838173
[ 0.41064435 -0.53086008]
0.671149168967
[ 0.1988391  -1.14387105]
1.16102452818
[-0.45195935 -1.58941787]
1.65242743456
[ 0.16073801 -1.27533488]
1.28542435073
[-0.20045944 -1.38208294]
1.3

[-0.38838079 -0.16037613]
0.420190601206
[-0.10589825 -0.5033264 ]
0.514346093121
[ 0.33738669 -1.3773226 ]
1.41804348249
[ 0.19239143 -1.4207343 ]
1.43370164319
[-0.12220373 -1.16356526]
1.16996489727
[ 0.1182284  -1.37683514]
1.38190193776
[-0.11807309 -1.28821039]
1.29361016442
[-0.03694342 -1.44029006]
1.44076378411
[ 0.19949527 -0.47313493]
0.513473487805
[-0.13932704 -1.15492007]
1.16329376578
[-0.92855128 -0.80703419]
1.23024861678
[ 0.14515195 -1.4421547 ]
1.44944101459
[-0.35054367 -1.23774385]
1.28642555696
[ 0.21769359 -0.86216851]
0.889227215224
[ 0.87826984 -1.33166239]
1.59520614453
[ 0.55358046 -1.49638066]
1.59549565618
[ 0.56290552 -0.99492647]
1.1431278593
[ 0.88505531 -0.62705963]
1.08467814334
[ 0.72873635 -1.26613361]
1.46087336587
[ 0.93126065 -1.07228251]
1.42022398857
[ 0.40553315 -1.09187225]
1.16474982271
[-0.42783448 -0.86529307]
0.965284645164
[-0.07525747 -1.00646156]
1.00927129624
[-0.57282721 -1.61183618]
1.71059839576
[ 0.20988476 -1.24189539]
1.25950615

[ 0.11237985 -0.97219306]
0.978666729726
[ 0.02151155 -0.84538355]
0.845657195028
[-0.20966044 -0.84244129]
0.868138718943
[ 0.3490538  -1.18659575]
1.23687025952
[ 0.44588964 -1.28180617]
1.35714576247
[ 0.29776666 -1.11000387]
1.14924913379
[-0.1195566  -1.41482567]
1.41986811077
[ 0.099588   -1.35051422]
1.35418109518
[-0.24527691 -0.79051669]
0.827694031312


KeyboardInterrupt: 